# Using OpenAI Latest Embeddings In A RAG System With MongoDB

This project explores the step-by-step implementation process of utilizing one of the new embedding models: text-embedding-3-small within a Retrieval Augmented Generation(RAG) System powered by MongoDB Atlas Vector Database.


## Step 1: Libraries Installation


Below are brief explanations of the tools and libraries utilised within the implementation code:
* **datasets**: This library is part of the Hugging Face ecosystem. By installing 'datasets', we gain access to a number of pre-processed and ready-to-use datasets, which are essential for training and fine-tuning machine learning models or benchmarking their performance.

* **pandas**: A data science library that provides robust data structures and methods for data manipulation, processing and analysis.

* **openai**: This is the official Python client library for accessing OpenAI's suite of AI models and tools, including GPT and embedding models.italicised text

* **pymongo**: PyMongo is a Python toolkit for MongoDB. It enables interactions with a MongoDB database.

In [ ]:
!pip install datasets pandas openai pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


## Step 2: Data Loading


Load the dataset titled ["AIatMongoDB/embedded_movies"](https://huggingface.co/datasets/AIatMongoDB/embedded_movies). This dataset is a collection of movie-related details that include attributes such as the title, release year, cast, plot and more. A unique feature of this dataset is the plot_embedding field for each movie. These embeddings are generated using OpenAI's text-embedding-ada-002 model.


In [ ]:
# 1. Load Dataset
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset['train'])
dataset_df.to_csv('movie.csv', index = False)

dataset_df.head(5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

,title,plot,cast,type,num_mflix_comments,plot_embedding,genres,runtime,poster,languages,writers,awards,countries,rated,metacritic,fullplot,imdb,directors
0,The Perils of Pauline,Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",movie,0,"[0.00072939653, -0.026834568, 0.013515796, -0....",[Action],199.0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[USA],None,NaN,Young Pauline is left a lot of money when her ...,"{'id': 4465, 'rating': 7.6, 'votes': 744}","[Louis J. Gasnier, Donald MacKenzie]"
1,From Hand to Mouth,A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",movie,0,"[-0.022837115, -0.022941574, 0.014937485, -0.0...","[Comedy, Short, Action]",22.0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",[USA],TV-G,NaN,As a penniless man worries about how he will m...,"{'id': 10146, 'rating': 7.0, 'votes': 639}","[Alfred J. Goulding, Hal Roach]"
2,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",movie,0,"[0.00023330493, -0.028511643, 0.014653289, -0....","[Action, Adventure, Drama]",101.0,None,[English],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[USA],None,NaN,"Michael ""Beau"" Geste leaves England in disgrac...","{'id': 16634, 'rating': 6.9, 'votes': 222}",[Herbert Brenon]
3,The Black Pirate,"Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",movie,1,"[-0.005927917, -0.033394486, 0.0015323418, -0....","[Adventure, Action]",88.0,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[USA],None,NaN,A nobleman vows to avenge the death of his fat...,"{'id': 16654, 'rating': 7.2, 'votes': 1146}",[Albert Parker]
4,For Heaven's Sake,An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",movie,0,"[-0.0059373598, -0.026604708, -0.0070914757, -...","[Action, Comedy, Romance]",58.0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",[USA],PASSED,NaN,"The Uptown Boy, J. Harold Manners (Lloyd) is a...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[Sam Taylor]


## Step 3: Data Cleaning and Preparation

The next step cleans the data and prepares it for the next stage, which creates a new embedding data point using the new OpenAI embedding model.


In [ ]:
print("Columns:", dataset_df.columns)
print("\nNumber of rows and columns:", dataset_df.shape)
print("\nBasic Statistics for numerical data:")
print(dataset_df.describe())
print("\nNumber of missing values in each column:")
print(dataset_df.isnull().sum())

Columns: Index(['cast', 'imdb', 'plot_embedding', 'poster', 'metacritic',
       'num_mflix_comments', 'countries', 'fullplot', 'title', 'plot',
       'languages', 'runtime', 'writers', 'rated', 'awards', 'genres',
       'directors', 'type'],
      dtype='object')

Number of rows and columns: (1500, 18)

Basic Statistics for numerical data:
       metacritic  num_mflix_comments      runtime
count  572.000000         1500.000000  1485.000000
mean    51.646853            6.071333   111.977104
std     16.861996           27.378982    42.090386
min      9.000000            0.000000     6.000000
25%     40.000000            0.000000    96.000000
50%     51.000000            0.000000   106.000000
75%     63.000000            1.000000   121.000000
max     97.000000          158.000000  1256.000000

Number of missing values in each column:
cast                    1
imdb                    0
plot_embedding         28
poster                 89
metacritic            928
num_mflix_comments      

In [ ]:
# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset=['plot'])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with the new OpenAI emebedding Model "text-embedding-3-small"
dataset_df = dataset_df.drop(columns=['plot_embedding'])
dataset_df.head(5)
dataset_df.shape


Number of missing values in each column after removal:
cast                    1
imdb                    0
plot_embedding          1
poster                 78
metacritic            903
num_mflix_comments      0
countries               0
fullplot               21
title                   0
plot                    0
languages               1
runtime                14
writers                13
rated                 284
awards                  0
genres                  0
directors              13
type                    0
dtype: int64


(1473, 17)

## Step 4: Create embeddings with OpenAI

This stage focuses on generating new embeddings using OpenAI's advanced model.
This demonstration utilises a Google Colab Notebook, where environment variables are configured explicitly within the notebook's Secret section and accessed using the user data module. In a production environment, the environment variables that store secret keys are usually stored in a '.env' file or equivalent.

An [OpenAI API](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key) key is required to ensure the successful completion of this step. More details on OpenAI's embedding models can be found on the official [site](https://platform.openai.com/docs/guides/embeddings).


In [ ]:
import openai
from google.colab import userdata

openai.api_key = "your key"

EMBEDDING_MODEL = "text-embedding-3-small"

def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

dataset_df["plot_embedding_optimised"] = dataset_df['plot'].apply(get_embedding)

dataset_df.head()

Error in get_embedding: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-RYcX3iagf8ACcnJxkK5W14YE on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Error in get_embedding: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-RYcX3iagf8ACcnJxkK5W14YE on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


KeyboardInterrupt: 

## Step 5: Vector Database Setup and Data Ingestion

MongoDB acts as both an operational and a vector database. It offers a database solution that efficiently stores, queries and retrieves vector embeddings—the advantages of this lie in the simplicity of database maintenance, management and cost.

**To create a new MongoDB database, set up a database cluster:**

1. Head over to MongoDB official site and register for a [free MongoDB Atlas account](https://www.mongodb.com/cloud/atlas/register), or for existing users, [sign into MongoDB Atlas](https://account.mongodb.com/account/login?nds=true).

2. Select the 'Database' option on the left-hand pane, which will navigate to the Database Deployment page, where there is a deployment specification of any existing cluster. Create a new database cluster by clicking on the "+Create" button.

3.   Select all the applicable configurations for the database cluster. Once all the configuration options are selected, click the “Create Cluster” button to deploy the newly created cluster. MongoDB also enables the creation of free clusters on the “Shared Tab”.

 *Note: Don’t forget to whitelist the IP for the Python host or 0.0.0.0/0 for any IP when creating proof of concepts.*

4. After successfully creating and deploying the cluster, the cluster becomes accessible on the ‘Database Deployment’ page.

5. Click on the “Connect” button of the cluster to view the option to set up a connection to the cluster via various language drivers.

6. This tutorial only requires the cluster's URI(unique resource identifier). Grab the URI and copy it into the Google Colabs Secrets environment in a variable named `MONGO_URI` or place it in a .env file or equivalent.




In [ ]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
  """Establish connection to the MongoDB."""
  try:
    client = pymongo.MongoClient(mongo_uri)
    print("Connection to MongoDB successful")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed: {e}")
    return None

mongo_uri = "YOUR URL"
if not mongo_uri:
  print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client['movies']
collection = db['movie_collection']

Connection to MongoDB successful


In [ ]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000121'), 'opTime': {'ts': Timestamp(1710943975, 14), 't': 289}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1710943975, 14), 'signature': {'hash': b'\xbb\xb4\x8c\xbaH\xbd\x80\xf7Yk\xf5\x155\xca\xf4\xb3\xf9I\xfd\x12', 'keyId': 7284051696911646722}}, 'operationTime': Timestamp(1710943975, 14)}, acknowledged=True)

In [ ]:
documents = dataset_df.to_dict('records')
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## Step 6: Create a Vector Search Index

At this point make sure that your vector index is created via MongoDB Atlas.
Follow instructions here:

This next step is mandatory for conducting efficient and accurate vector-based searches based on the vector embeddings stored within the documents in the ‘movie_collection’ collection. Creating a Vector Search Index enables the ability to traverse the documents efficiently to retrieve documents with embeddings that match the query embedding based on vector similarity. Go here to read more about [MongoDB Vector Search Index](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector/).


## Step 7: Perform Vector Search on User Queries

This step combines all the activities in the previous step to provide the functionality of conducting vector search on stored records based on embedded user queries.

This step implements a function that returns a vector search result by generating a query embedding and defining a MongoDB aggregation pipeline. The pipeline, consisting of the `$vectorSearch` and `$project` stages, queries using the generated vector and formats the results to include only required information like plot, title, and genres while incorporating a search score for each result.

This selective projection enhances query performance by reducing data transfer and optimizes the use of network and memory resources, which is especially critical when handling large datasets. For AI Engineers and Developers considering data security at an early stage, the chances of sensitive data leaked to the client side can be minimized by carefully excluding fields irrelevant to the user's query.


In [ ]:

def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "plot_embedding_optimised",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 5  # Return top 5 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "plot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1, # Include the genres field
                "score": {
                    "$meta": "vectorSearchScore"  # Include the search score
                }
            }
        }
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

## Step 8: Handling User Query and Result

The final step in the implementation phase focuses on the practical application of our vector search functionality and AI integration to handle user queries effectively.

The handle_user_query function performs a vector search on the MongoDB collection based on the user's query and utilizes OpenAI's GPT-3.5 model to generate context-aware responses.


In [ ]:
def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:
      search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('plot', 'N/A')}\n"

  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a movie recommendation system."},
          {"role": "user", "content": "Answer this user query: " + query + " with the following context: " + search_result}
      ]
  )

  return (completion.choices[0].message.content), search_result

In [ ]:
# 6. Conduct query with retrival of sources
query = "What are the best action movie?"
response, source_information = handle_user_query(query, collection)

print(f"Response: {response}")
print(f"Source Information: \n{source_information}")


Response: Certainly! Here are some of the best action movies that are highly acclaimed and popular among audiences:

1. "Mad Max: Fury Road" (2015) - A high-octane post-apocalyptic action film that is visually stunning with intense action sequences.

2. "The Dark Knight" (2008) - Christopher Nolan's gritty and realistic take on the Batman franchise, with Heath Ledger's iconic performance as the Joker.

3. "Die Hard" (1988) - A classic action film starring Bruce Willis as a New York cop battling terrorists in a Los Angeles skyscraper.

4. "John Wick" (2014) - Keanu Reeves stars as a retired hitman seeking vengeance in this stylish and action-packed film.

5. "The Matrix" (1999) - A groundbreaking sci-fi action film that redefined the genre with its innovative visual effects and fight choreography.

These movies offer a mix of adrenaline-pumping action, compelling characters, and engaging storylines that have made them favorites among action movie fans.
Source Information: 



In [ ]:
!npm install localtunnel
!pip install streamlit

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.742s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile app.py
import openai
from google.colab import userdata
import pymongo
import streamlit as st

openai.api_key = "API KEY"
EMBEDDING_MODEL = "text-embedding-3-small"


def get_mongo_client(mongo_uri):
  """Establish connection to the MongoDB."""
  try:
    client = pymongo.MongoClient(mongo_uri)
    print("Connection to MongoDB successful")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed: {e}")
    return None

mongo_uri = "MONGO URL"
if not mongo_uri:
  print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client['movies']
collection = db['movie_collection']

# Delete any existing records in the collection
def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "plot_embedding_optimised",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 5  # Return top 5 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "plot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1, # Include the genres field
                "score": {
                    "$meta": "vectorSearchScore"  # Include the search score
                }
            }
        }
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:
      search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('plot', 'N/A')}\n"

  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a movie recommendation system."},
          {"role": "user", "content": "Answer this user query: " + query + " with the following context: " + search_result}
      ]
  )

  return (completion.choices[0].message.content), search_result

# 6. Conduct query with retrival of sources

st.set_page_config(page_title="Movie Recommendation")
st.header("Ask For Movie Recommendation")
query = st.text_input("Query")
if query != "":
  response, source_information = handle_user_query(query, collection)
  print(source_information)
  print(response)
  st.write(f"Response: {response}\n\nSource Information: {source_information}")






Writing app.py


In [ ]:
!npx localtunnel --port 8501 & streamlit run app.py & curl ipv4.icanhazip.com
